# Titanic - Machine Learning from Disaster

## Features constructs

> Создать новые переменные из Name

In [1]:
import re

import numpy as np
import pandas as pd

## Импорт данных

In [2]:
df = pd.read_csv('data/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Анализ данных

### Описание данных

| Variable | Definition                                  | Key                                            |
|----------|---------------------------------------------|------------------------------------------------|
| survival | Survival	                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                                | 	1 = 1st, 2 = 2nd, 3 = 3rd                     |
| sex	     | Sex                                         |                                                |	
| Age      | 	Age in years                               |                                                |	
| sibsp    | 	# of siblings / spouses aboard the Titanic |                                                |	
| parch    | 	# of parents / children aboard the Titanic |                                                |	
| ticket   | 	Ticket number                              |                                                |	
| fare     | 	Passenger fare                             |                                                |	
| cabin    | 	Cabin number                               |                                                |	
| embarked | 	Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes
**pclass**: A proxy for socio-economic status (SES)

1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...

**Sibling** = brother, sister, stepbrother, stepsister

**Spouse** = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...

**Parent** = mother, father

**Child** = daughter, son, stepdaughter, stepson

**Some** children travelled only with a nanny, therefore parch=0 for them.


In [3]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
PassengerId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Survived,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1,0,1
Pclass,3,1,3,1,3,3,1,3,3,2,3,1,3,3,3,2,3,2,3,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry","Moran, Mr. James","McCarthy, Mr. Timothy J","Palsson, Master. Gosta Leonard","Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)","Nasser, Mrs. Nicholas (Adele Achem)","Sandstrom, Miss. Marguerite Rut","Bonnell, Miss. Elizabeth","Saundercock, Mr. William Henry","Andersson, Mr. Anders Johan","Vestrom, Miss. Hulda Amanda Adolfina","Hewlett, Mrs. (Mary D Kingcome)","Rice, Master. Eugene","Williams, Mr. Charles Eugene","Vander Planke, Mrs. Julius (Emelia Maria Vande...","Masselmani, Mrs. Fatima"
Sex,male,female,female,female,male,male,male,male,female,female,female,female,male,male,female,female,male,male,female,female
Age,22.0,38.0,26.0,35.0,35.0,NaN,54.0,2.0,27.0,14.0,4.0,58.0,20.0,39.0,14.0,55.0,2.0,NaN,31.0,NaN
SibSp,1,1,0,1,0,0,0,3,0,1,1,0,0,1,0,0,4,0,1,0
Parch,0,0,0,0,0,0,0,1,2,0,1,0,0,5,0,0,1,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450,330877,17463,349909,347742,237736,PP 9549,113783,A/5. 2151,347082,350406,248706,382652,244373,345763,2649
Fare,7.25,71.2833,7.925,53.1,8.05,8.4583,51.8625,21.075,11.1333,30.0708,16.7,26.55,8.05,31.275,7.8542,16.0,29.125,13.0,18.0,7.225


## Обработка данных

### Разделение на выборки 

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
target = df['Survived']
# features = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1)
features = df.drop(['PassengerId', 'Ticket', 'Cabin', 'Survived'], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)
X_train.shape, X_test.shape

((596, 8), (295, 8))

In [7]:
X_train.head(2)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,51.8625,S
718,3,"McEvoy, Mr. Michael",male,NaN,0,0,15.5000,Q


## Создание конвейера

План:

1. Численные:
 - Age - заполнение пропусков
2. Категорийный:
 - Embarked, Sex - Кодирование

In [8]:
import pandas as pd
from sklearn.compose import ColumnTransformer

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier


### Итоговый конвейер

In [9]:

X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,51.8625,S
718,3,"McEvoy, Mr. Michael",male,NaN,0,0,15.5000,Q
685,2,"Laroche, Mr. Joseph Philippe Lemercier",male,25.0,1,2,41.5792,C
73,3,"Chronopoulos, Mr. Apostolos",male,26.0,1,0,14.4542,C
882,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,10.5167,S


In [10]:
def get_title(df, col):
    df[col] = df[col].str.extract(r'.*\, ([a-zA-Z]*\.) ')
    return df


In [11]:
encoder_ord = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
pipe_name = Pipeline([('regexp_title', FunctionTransformer(get_title, kw_args={'col': 'Name'})),
                      ('encoder_ord', encoder_ord), ])
pipe_imputer = Pipeline([('knn', KNNImputer())])

pipe_preproc = ColumnTransformer(
    transformers=[('encoder', pipe_name, ['Name']), ('cat_preproc', encoder_ord, ['Embarked', 'Sex'])],
    remainder='passthrough',
    force_int_remainder_cols=False).set_output(transform='pandas')

model = DecisionTreeClassifier(random_state=0)

pipe_process = Pipeline([('preproc', pipe_preproc), ('imputer', pipe_imputer)])
pipe_model = Pipeline([('process', pipe_process), ('model', model)])


In [12]:
# pipe_model.set_params(model__max_depth=7, model__max_features=5, process__imputer__knn__n_neighbors=9)
#{'model__max_depth': 6, 'model__max_features': 5, 'process__imputer__knn__n_neighbors': 7}
pipe_model.set_params(model__max_depth=6, model__max_features=4, process__imputer__knn__n_neighbors=3)

Pipeline(steps=[('process',
                 Pipeline(steps=[('preproc',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('encoder',
                                                                   Pipeline(steps=[('regexp_title',
                                                                                    FunctionTransformer(func=<function get_title at 0x0000022F4A653CE0>,
                                                                                                        kw_args={'col': 'Name'})),
                                                                                   ('encoder_ord',
                                                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                   unknown_value=nan))]),
                                                                   ['Name']),
                                                                  ('cat_preproc',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan),
                                                                   ['Embarked',
                                                                    'Sex'])])),
                                 ('imputer',
                                  Pipeline(steps=[('knn',
                                                   KNNImputer(n_neighbors=3))]))])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, max_features=4,
                                        random_state=0))])

In [13]:
pipe_model.fit(X_train, y_train)

Pipeline(steps=[('process',
                 Pipeline(steps=[('preproc',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('encoder',
                                                                   Pipeline(steps=[('regexp_title',
                                                                                    FunctionTransformer(func=<function get_title at 0x0000022F4A653CE0>,
                                                                                                        kw_args={'col': 'Name'})),
                                                                                   ('encoder_ord',
                                                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                   unknown_value=nan))]),
                                                                   ['Name']),
                                                                  ('cat_preproc',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan),
                                                                   ['Embarked',
                                                                    'Sex'])])),
                                 ('imputer',
                                  Pipeline(steps=[('knn',
                                                   KNNImputer(n_neighbors=3))]))])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, max_features=4,
                                        random_state=0))])

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(y_train, pipe_model.predict(X_train))

0.860738255033557

In [16]:
accuracy_score(y_test, pipe_model.predict(X_test))

0.7932203389830509

In [67]:

from sklearn.model_selection import GridSearchCV

In [89]:
model_params = {'model__max_depth': range(1, 11), 'model__max_features': range(4, 19),
                'process__imputer__knn__n_neighbors': range(3, 10)}
tree_grid = GridSearchCV(pipe_model, model_params, cv=10, n_jobs=-1, verbose=True)

In [90]:
tree_grid.fit(X_train, y_train)


Fitting 10 folds for each of 1050 candidates, totalling 10500 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('process',
                                        Pipeline(steps=[('preproc',
                                                         ColumnTransformer(force_int_remainder_cols=False,
                                                                           remainder='passthrough',
                                                                           transformers=[('encoder',
                                                                                          Pipeline(steps=[('regexp_title',
                                                                                                           FunctionTransformer(func=<function get_title at 0x0000020BF575CC20>,
                                                                                                                               kw_args={'col': 'Name'})),
                                                                                                          ('encoder_ord',
                                                                                                           OrdinalEncoder(handle_unknown='u...
                                                                                          OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                         unknown_value=nan),
                                                                                          ['Embarked',
                                                                                           'Sex'])])),
                                                        ('imputer',
                                                         Pipeline(steps=[('knn',
                                                                          KNNImputer(n_neighbors=3))]))])),
                                       ('model',
                                        DecisionTreeClassifier(max_depth=6,
                                                               max_features=4,
                                                               random_state=0))]),
             n_jobs=-1,
             param_grid={'model__max_depth': range(1, 11),
                         'model__max_features': range(4, 19),
                         'process__imputer__knn__n_neighbors': range(3, 10)},
             verbose=True)

In [91]:
tree_grid.best_params_

{'model__max_depth': 6,
 'model__max_features': 5,
 'process__imputer__knn__n_neighbors': 7}

### Метрики обучения

> Скор на трейне : 0.979
> Скор на валиде : 0.76
> Вывод достигли переобучение

### Исследование pipeline

In [26]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,male,54.0,0,0,51.8625,S
718,3,male,NaN,0,0,15.5000,Q


In [78]:
pipe_model.steps

[('process',
  Pipeline(steps=[('preproc',
                   ColumnTransformer(force_int_remainder_cols=False,
                                     remainder='passthrough',
                                     transformers=[('encoder',
                                                    Pipeline(steps=[('regexp_title',
                                                                     FunctionTransformer(func=<function get_title at 0x0000020BF575CC20>,
                                                                                         kw_args={'col': 'Name'})),
                                                                    ('encoder_ord',
                                                                     OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                    unknown_value=nan))]),
                                                    ['Name']),
                                                  

In [79]:
pipe_model.named_steps['preproc1'].named_transformers_['cat_preproc'].named_steps['encoder'].categories_

KeyError: 'preproc1'

In [97]:
pipe_model.named_steps['model'].feature_importances_

array([0.25077015, 0.07209911, 0.30282177, 0.10209646, 0.06553953,
       0.04746769, 0.03851653, 0.12068876])

In [98]:
X_train.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

Максимальное влияние параметров:
1. Sex - 0.29
2. SibSp (наличие родственников) - 0.27
3. Embarked (порт назначения) - 0.20


### Кроссвалидация

In [82]:

from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(pipe_model, X_train, y_train, cv=5,
                             scoring='accuracy')

In [83]:
cv_results

array([0.75833333, 0.8907563 , 0.83193277, 0.79831933, 0.82352941])

## Предсказание на реальных данных

In [111]:
df_ground = pd.read_csv('data/test.csv')
df_ground.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [112]:
df_submission = pd.read_csv('data/gender_submission.csv')
df_submission.head(2)

,PassengerId,Survived
0,892,0
1,893,1


In [113]:
df_ground['Survived'] = pipe_model.predict(df_ground.drop('PassengerId', axis=1))


In [114]:
df_ground

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [115]:
df_ground[['PassengerId', 'Survived']].to_csv('data/test7.csv', index=False)

In [116]:
! kaggle competitions submit titanic -f .\data\test7.csv -m"title "

Successfully submitted to Titanic - Machine Learning from Disaster



  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|##########| 3.18k/3.18k [00:00<00:00, 7.26kB/s]


### Метрика на лидерборде

Score: 0.73923

place: 12189

После оптимизации knn

Score: 0.74880

place: 11928

Можно сказать, что улучшение качества модели существенно улучшило результат на лидерборде.




Score: 0.79665
Place: 909